In [22]:
%load_ext autoreload
%autoreload 2

In [52]:
from datapipeline import Datapipeline
import pandas as pd
import numpy as np

In [9]:
dpl = Datapipeline()

In [16]:
import os
print(os.getcwd())
print(os.path.dirname(os.getcwd()))

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode
d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5


In [71]:
raw_data_path = 'd:\\Users\mokky\\Documents\\GitHub\\nus-iss\\PRS-PM-ISY5002-GROUP5\\SystemCode\\Data Exploration\\data\\ParkwaySampleDataForProject_09_withTOSP3.xlsx'
dict_hosp_filepaths = dpl.transform_raw_data(raw_data_path)

In [38]:
print(dict_hosp_filepaths)

{&#39;GHL&#39;: &#39;./data/GHL_data.csv&#39;, &#39;MEH&#39;: &#39;./data/MEH_data.csv&#39;, &#39;PEH&#39;: &#39;./data/PEH_data.csv&#39;, &#39;PNH&#39;: &#39;./data/PNH_data.csv&#39;}


In [47]:
dict_hosp_df = {}
for hosp in dict_hosp_filepaths:
    dict_hosp_df[hosp] = pd.read_csv(dict_hosp_filepaths[hosp])
    print(len(dict_hosp_df[hosp]))

106229
131062
38138
105781


In [66]:
df = pd.read_csv('./data/all_hosp_data.csv')
df = dpl.convert_numeric_to_object(df)
print(df.info())

[&#39;SPECIALTY_CODE&#39;, &#39;SPECIALTY_GRP&#39;, &#39;PACKAGE_CODE&#39;, &#39;PACKAGE_CODE1&#39;, &#39;PACKAGE_CODE2&#39;]
&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 381210 entries, 0 to 381209
Data columns (total 56 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CASE_NUMBER         381210 non-null  int64  
 1   INSTITUTION         381210 non-null  object 
 2   TOTAL_PAID_AMT      381210 non-null  float64
 3   PAYER_CODE_1        381210 non-null  object 
 4   PAYER_1_PAID_AMT    381210 non-null  float64
 5   PAYER_CODE_2        149318 non-null  object 
 6   PAYER_2_PAID_AMT    381210 non-null  float64
 7   PAYER_CODE_3        53205 non-null   object 
 8   PAYER_3_PAID_AMT    381210 non-null  float64
 9   PAYER_CODE_4        2373 non-null    object 
 10  PAYER_4_PAID_AMT    381210 non-null  float64
 11  PAYER_CODE_5        52 non-null      object 
 12  PAYER_5_PAID_AMT    381210 non-null  float64

In [67]:
print(df.select_dtypes(include=np.number).isna().sum())

CASE_NUMBER              0
TOTAL_PAID_AMT           0
PAYER_1_PAID_AMT         0
PAYER_2_PAID_AMT         0
PAYER_3_PAID_AMT         0
PAYER_4_PAID_AMT         0
PAYER_5_PAID_AMT         0
DISCHARGE_TYPE           0
LOS_DAYS                 0
TOSP_COUNT               0
PATIENT_NUMBER           0
PACKAGE_PRICE       290111
PACKAGE_EXCL        290111
PACKAGE_ADJ         290111
PROF_FEE                 0
TOTAL_FEES               0
WRITE_OFF                0
Admission_Age            0
dtype: int64


In [68]:
print('CASE_NUMBER vs PATIENT_NUMBER: ',
      df.groupby(by=['CASE_NUMBER'])['PATIENT_NUMBER'].nunique().max())

CASE_NUMBER vs PATIENT_NUMBER:  1


In [80]:
print(df.select_dtypes(include=np.object).columns[df.select_dtypes(include=np.object).isnull().any()].tolist())
print(df.select_dtypes(include=np.object).isnull().sum())

[&#39;PAYER_CODE_2&#39;, &#39;PAYER_CODE_3&#39;, &#39;PAYER_CODE_4&#39;, &#39;PAYER_CODE_5&#39;, &#39;REFERRAL_TYPE&#39;, &#39;DOCTOR_CODE&#39;, &#39;SPECIALTY_CODE&#39;, &#39;SPECIALTY_GRP&#39;, &#39;TOSP_CODE1&#39;, &#39;TOSP_CODE2&#39;, &#39;TOSP_CODE3&#39;, &#39;TOSP_CODE4&#39;, &#39;MARITAL_STATUS&#39;, &#39;RELIGION&#39;, &#39;RACE&#39;, &#39;DRG_CODE&#39;, &#39;PAYER_CODE2_V&#39;, &#39;PAYER_CODE3_V&#39;, &#39;PAYER_CODE4_V&#39;, &#39;PACKAGE_CODE1&#39;, &#39;PACKAGE_CODE2&#39;, &#39;ICD_CODE1&#39;, &#39;ICD_CODE2&#39;, &#39;ICD_CODE3&#39;, &#39;Resid_Cty_Code&#39;]
INSTITUTION                0
PAYER_CODE_1               0
PAYER_CODE_2          231892
PAYER_CODE_3          328005
PAYER_CODE_4          378837
PAYER_CODE_5          381158
BED_TYPE                   0
REFERRAL_TYPE              4
TREATMENT_CATEGORY         0
ADMISSION_DTE              0
ADMISSION_TYPE             0
DOCTOR_CODE                1
SPECIALTY_CODE            37
SPECIALTY_GRP           8319
TOSP_CODE1    

In [59]:
def _bin_column(df, col, bin_thresh):
    bin_intervals = bin_thresh[:]
    bin_intervals.insert(0, np.NINF)
    bin_intervals.append(np.Inf)
    df[f'bin_{col}'] = pd.cut(df[col], bin_intervals)
    return df

bin_thresh = [0, 100, 500, 1000, 5000, 10000]
df = _bin_column(df, 'WRITE_OFF', bin_thresh)
print(df['bin_WRITE_OFF'].value_counts())

bin_thresh = [0, 100, 1000, 10000]
df = _bin_column(df, 'WRITE_OFF', bin_thresh)
print(df['bin_WRITE_OFF'].value_counts())

bin_thresh = [0, 100, 1000]
df = _bin_column(df, 'WRITE_OFF', bin_thresh)
print(df['bin_WRITE_OFF'].value_counts())

bin_thresh = [0]
df = _bin_column(df, 'WRITE_OFF', bin_thresh)
print(df['bin_WRITE_OFF'].value_counts())

(-inf, 0.0]          369685
(0.0, 100.0]          10697
(100.0, 500.0]          316
(1000.0, 5000.0]        260
(5000.0, 10000.0]        85
(500.0, 1000.0]          85
(10000.0, inf]           82
Name: bin_WRITE_OFF, dtype: int64
(-inf, 0.0]          369685
(0.0, 100.0]          10697
(100.0, 1000.0]         401
(1000.0, 10000.0]       345
(10000.0, inf]           82
Name: bin_WRITE_OFF, dtype: int64
(-inf, 0.0]        369685
(0.0, 100.0]        10697
(1000.0, inf]         427
(100.0, 1000.0]       401
Name: bin_WRITE_OFF, dtype: int64
(-inf, 0.0]    369685
(0.0, inf]      11525
Name: bin_WRITE_OFF, dtype: int64


In [ ]:
df_sample1 = get_best_sample(df1, sample_frac=0.1)
print(df_sample1.shape)